In [21]:
import requests
from bs4 import BeautifulSoup
from sympy import per
import yfinance as yf
import pandas as pd
import numpy as np

stocks = ['BTU', 'NEX', 'OXY', 'KR', 'XOM', 'WMT', 'JNJ', 'KO', 'GOOGL', 'MA', 'AAPL', 'DDOG', 'PBF', 'AMZN', 'TSLA',
         'PEP', 'MSFT', 'MCD', 'NVDA', 'GS', 'JPM', 'SQ', 'PLTR', 'ROKU', 'TDOC']


PFA, v, score, stockslist, industry, market_cap, short_float, price, beta = [], [], [], [], [], [], [], [], []

for stock in stocks:
    ticker = yf.Ticker(stock)
    
    #Current Price
    try:
        current_price = ticker.info['currentPrice']
    except:
        current_price = 0
        
    #52-Week High
    try:
        ATH = ticker.info['fiftyTwoWeekHigh']
    except:
        ATH = 1
    percent_from_52 = round((1 - current_price/ATH)*100, 2)
    
    if percent_from_52 == 100:
        PFA.append(0)
    else:
         PFA.append(percent_from_52)
    
    #Volatility
    url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    try:
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = float(volatility.text)*100
        vol = round(vol, 2)
        if vol == 0:
            v.append("No data")
        else:
            v.append(vol)
    except:
        vol = "No Data"
        v.append(vol)
    
    #Market Cap
    try:
        mkcp = ticker.info['marketCap']
        market_cap.append(round(mkcp/1000000000))
    except:
        mkcp = 0
        market_cap.append("No data")
 
    #Industry
    try:
        industry.append(ticker.info['sector'])
    except:
        industry.append("No data")
    
    #Short Float
    try:
        shorts = ticker.info['shortPercentOfFloat']
        short_float.append(round(float(shorts*100), 2))
    except:
        shorts = 0
        short_float.append("No data")
    
    #Strength Score
    if mkcp == 0 or vol == 0:
        score.append(0)
    else:
        strength_score = ((0-percent_from_52 + (vol/2))/2 + (shorts/5))/2
        score.append(round(strength_score, 1))
        
    stockslist.append(stock)    
    price.append('$' + str(round(current_price, 2)))
        
data = {'Stock': stockslist,
        'Industry': industry,
        'Current Price': price,
        'Market Cap (B)': market_cap,
        '% Float Short': short_float,
        '% From 52w High': PFA,
        'Volatility': v,
        'Strength Score': score}

df = pd.DataFrame(data)
df = df.sort_values(by='Strength Score', ascending=False)
df.reset_index(drop = True, inplace=True)
df.index = np.arange(1, len(df)+1)
df

#Current date: 4/26/22

,Stock,Industry,Current Price,Market Cap (B),% Float Short,% From 52w High,Volatility,Strength Score
1,BTU,Energy,$26.02,3,8.58,21.84,101.37,7.2
2,PBF,Energy,$26.91,3,22.09,10.70,75.73,6.8
3,NEX,Energy,$10.48,3,2.58,11.37,73.48,6.3
4,OXY,Energy,$55.95,52,5.72,11.98,58.71,4.3
5,KR,Consumer Defensive,$55.99,41,4.16,10.82,39.09,2.2
6,JNJ,Healthcare,$185.5,488,0.58,0.64,17.95,2.1
7,XOM,Energy,$83.61,356,1.08,8.63,31.14,1.7
8,WMT,Consumer Defensive,$155.71,432,0.89,3.15,19.15,1.6
9,PEP,Consumer Defensive,$173.64,240,0.88,2.03,16.55,1.6
10,KO,Consumer Defensive,$65.38,282,0.73,2.71,17.81,1.5
